In [ ]:
# automatically reload edited modules
%load_ext autoreload
%autoreload 2
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import matplotlib.image as im

import numpy as np
import glob
import os
import sys
import cv2
import glob

from scipy import io

# 데이터 불러오기

![alt text](https://ibug.doc.ic.ac.uk/media/uploads/images/ear-examplar.png "ears_data_set_image")

출처 : https://ibug.doc.ic.ac.uk/resources/ibug-ears/

In [ ]:
# data_dir = '../DATA_SET/Ear_data/train/o_images/'
# landmark_dir = '../DATA_SET/Ear_data/train/o_landmarks/'

data_dir = '../DATA_SET/300W-LP/300W_LP/AFW/'
landmark_dir = '../DATA_SET/300W-LP/300W_LP/landmarks/AFW/'

file_list = glob.glob(data_dir+'*_0.jpg')
file_list = map(os.path.basename, file_list)
# file_list = os.listdir(data_dir)
name_list = list(set(map(lambda x : x[:-4], file_list)))

def _read_mat(file_path):
    matfile = io.loadmat(file_path)
    label = (matfile['pts_2d']).astype(np.float32)
    return label

"""
for name in name_list[:1]:
    image = im.imread(os.path.join(data_dir, name+'.png'))
    print(image.shape)
    with open(landmark_dir+name+'.txt', 'r') as f:
        lines_list = f.readlines()
        temp = list(map(lambda l: list(map(float, l.split(' '))), lines_list[3:-1]))
        print(np.array(temp).shape)
        for line in lines_list[3:-1]:
            x, y = list(map(float,line.split(' ')))
            image = cv2.circle(image, (int(x), int(y)), 3, (1, 0, 0), -1)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()
"""

for name in name_list[:1]:
    image = im.imread(os.path.join(data_dir, name+'.jpg'))
    print(image.shape)

    label = _read_mat(landmark_dir+name+'_pts.mat')
    for x, y in label:
        cv2.circle(image, (int(x), int(y)), 3, (255, 0, 0), -1)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()


In [ ]:
# image resize 
for name in name_list[:1]:
    image = im.imread(os.path.join(data_dir, name+'.jpg'))
    print(image.shape)

    resized_image = tf.image.resize(image ,[int(image.shape[0]/4), int(image.shape[0]/2)])

    label = _read_mat(landmark_dir+name+'_pts.mat')
    for x, y in label:
        cv2.circle(image, (int(x), int(y)), 3, (255, 0, 0), -1)

    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()

    idx = np.array([image.shape[1]/2, image.shape[0]/2])
    norm_label = (label-idx)/idx
    idx = np.array([resized_image.shape[1]/2, resized_image.shape[0]/2])
    new_label = (norm_label*idx)+idx+int(image.shape[0]/4)

    str = int(image.shape[0]/4)
    dst = str + resized_image.shape[0]
    pad_image = np.zeros_like(image)
    pad_image[str:dst,str:dst,:] = resized_image
    
    for x, y in new_label:
        cv2.circle(pad_image, (int(x), int(y)), 3, (255, 0, 0), -1)

    plt.imshow(pad_image)
    plt.show()

In [ ]:
image.shape[0]/2

In [ ]:
landmark = []
for line in lines_list[3:-1]:
    x, y = list(map(float,line.split(' ')))
    landmark.append([int(x), int(y)])
landmark = np.array(landmark)
min_x = np.min(landmark[:,0])
max_x = np.max(landmark[:,0])
min_y = np.min(landmark[:,1])
max_y = np.max(landmark[:,1])
print(min_x, max_x, min_y, max_y)

image_left = image[:,0:min_x,:]
image_right = image[:,max_x:-1,:]
image_mid = image[:,min_x:max_x:3,:]

new_image = np.concatenate((image_left, image_mid, image_right), axis = 1)

plt.figure(figsize=(10, 10))
plt.imshow(new_image)
plt.show()

landmark = (landmark-np.array([min_x,0]))/np.array([3,1])+np.array([min_x,0])
for x,y in landmark:
    new_image = cv2.circle(new_image, (int(x), int(y)), 3, (0, 1, 0), -1)
plt.figure(figsize=(10, 10))
plt.imshow(new_image)
plt.show()

# 모델 설정 및 파라미터 설정

In [ ]:
INPUT_SIZE = 200
MAP_SIZE = 100
MAP_SIGMA = 2.5

LANDMARK_NUM = list(range(68))#[4,9,16,36] # 원하는 point 를 입력
LANDMARK_SIZE = len(LANDMARK_NUM) # ear : 55 / face : 68
LEARNING_RATE = 0.0001
BATCH_SIZE = 10
EPOCH = 200

split_rate = 0.9

In [ ]:
#-------------------------------------- model setting
#------------------------- feature map
x = tf.keras.Input(shape=(INPUT_SIZE, INPUT_SIZE, 3), name='input_layer')
h = tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding = 'same')(x)
h_1 = tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.MaxPool2D()(h_1)

h = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h_2 = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.MaxPool2D()(h_2)

h = tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h_3 = tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.MaxPool2D()(h_3)

h = tf.keras.layers.Conv2D(512, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(512, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(512, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(512, kernel_size=(3,3), activation='relu', padding = 'same')(h)

h = tf.keras.layers.Conv2D(512, kernel_size=(3,3), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(512, kernel_size=(3,3), activation='relu', padding = 'same')(h)
feature_map = tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu', padding = 'same', name='feature_map')(h)

#------------------------- stage1
h = tf.keras.layers.Conv2D(512, kernel_size=(1,1), activation='relu', name='stage_1')(feature_map)

h = tf.keras.layers.Conv2D(256, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_3])

h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_2])

s1 = tf.keras.layers.Conv2D(LANDMARK_SIZE, kernel_size=(1,1), name='s1')(h)

s1_h = tf.keras.layers.MaxPool2D((4,4))(s1)

#------------------------- stage2
h = tf.keras.layers.concatenate([feature_map, s1_h], axis=-1, name='stage_2')
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)

h = tf.keras.layers.Conv2D(256, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_3])

h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_2])

s2 = tf.keras.layers.Conv2D(LANDMARK_SIZE, kernel_size=(1,1), name='s2')(h)

s2_h = tf.keras.layers.MaxPool2D((4,4))(s2)

#------------------------- stage3
h = tf.keras.layers.concatenate([feature_map, s2_h], axis=-1, name='stage_3')
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)

h = tf.keras.layers.Conv2D(256, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_3])

h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_2])

s3 = tf.keras.layers.Conv2D(LANDMARK_SIZE, kernel_size=(1,1), name='s3')(h)

s3_h = tf.keras.layers.MaxPool2D((4,4))(s3)

#------------------------- stage4
h = tf.keras.layers.concatenate([feature_map, s3_h], axis=-1, name='stage_4')
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)

h = tf.keras.layers.Conv2D(256, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_3])

h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_2])

s4 = tf.keras.layers.Conv2D(LANDMARK_SIZE, kernel_size=(1,1), name='s4')(h)

s4_h = tf.keras.layers.MaxPool2D((4,4))(s4)

#------------------------- stage5
h = tf.keras.layers.concatenate([feature_map, s4_h], axis=-1, name='stage_5')
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)

h = tf.keras.layers.Conv2D(256, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_3])

h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_2])

s5 = tf.keras.layers.Conv2D(LANDMARK_SIZE, kernel_size=(1,1), name='s5')(h)

s5_h = tf.keras.layers.MaxPool2D((4,4))(s5)

#------------------------- stage6
h = tf.keras.layers.concatenate([feature_map, s5_h], axis=-1, name='stage_6')
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(7,7), activation='relu', padding = 'same')(h)
h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)

h = tf.keras.layers.Conv2D(256, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_3])

h = tf.keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu')(h)
h = tf.keras.layers.UpSampling2D()(h)
h = tf.keras.layers.Add()([h, h_2])

s6 = tf.keras.layers.Conv2D(LANDMARK_SIZE, kernel_size=(1,1), name='s6')(h)

#------------------------- return
output = h = tf.keras.layers.concatenate([s1, s2, s3, s4, s5, s6], axis=-1, name='output_layer')
#-------------------------------------- model setting end

def _stage_loss(y_true, y_pred):
    stage = 6
    #y_ture = tf.image.resize(y_true, size=[feat_size, feat_size])
    #threshold = 0.0001
    mask = y_true != 0 #> threshold
    resized_mask   = tf.tile(mask, [1,1,1,stage])
    resized_y_true = tf.tile(y_true, [1,1,1,stage])
    resized_y_pred = y_pred
    #resized_y_pred = tf.image.resize(y_pred, size=[INPUT_SIZE, INPUT_SIZE])
    resized_y_pred = tf.image.resize(y_pred, size=[MAP_SIZE, MAP_SIZE])
    
    loss = tf.math.reduce_mean(tf.math.square(resized_y_true - resized_y_pred) * tf.cast(resized_mask, tf.float32), axis=-1)
    #loss = tf.math.reduce_mean(tf.math.square(resized_y_true - resized_y_pred), axis=-1)
    
    #loss = tf.keras.losses.mean_squared_error(resized_y_true, resized_y_pred)
    return loss

model = tf.keras.Model(inputs=[x], outputs=[output])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=_stage_loss)

# model = keras.models.load_model('saved_model_openpose_ears_ver4', compile=False)
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=_stage_loss)
model.summary()

In [ ]:
def _stage_loss(y_true, y_pred):
    stage = 6
    #y_ture = tf.image.resize(y_true, size=[feat_size, feat_size])
    #threshold = 0.0001
    mask = y_true != 0 #> threshold
    resized_mask   = tf.tile(mask, [1,1,1,stage])
    resized_y_true = tf.tile(y_true, [1,1,1,stage])
    resized_y_pred = y_pred
    #resized_y_pred = tf.image.resize(y_pred, size=[INPUT_SIZE, INPUT_SIZE])
    resized_y_pred = tf.image.resize(y_pred, size=[MAP_SIZE, MAP_SIZE])
    
    loss = tf.math.reduce_mean(tf.math.square(resized_y_true - resized_y_pred) * tf.cast(resized_mask, tf.float32), axis=-1)
    #loss = tf.math.reduce_mean(tf.math.square(resized_y_true - resized_y_pred), axis=-1)
    
    #loss = tf.keras.losses.mean_squared_error(resized_y_true, resized_y_pred)
    return loss
model = keras.models.load_model('saved_model_openpose_face', compile=False)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=_stage_loss)

# 데이터 전처리 설정

In [ ]:
input_size_h = INPUT_SIZE/2
feat_size = model.get_layer('feature_map').output_shape[1]

#--------------------------------------

train_len = int(len(name_list)*split_rate)
test_len = len(name_list)-train_len
print(train_len, test_len)

train_dataset = tf.data.Dataset.from_tensor_slices(name_list[:train_len])
test_dataset = tf.data.Dataset.from_tensor_slices(name_list[train_len:])

def process_path(name):
    image_path = data_dir+name+'.jpg'
    #image_path = data_dir+name+'.png'
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image)
    #image = tf.image.decode_png(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.cond(tf.shape(image)[-1] != 3,
                    lambda: tf.image.grayscale_to_rgb(image),
                    lambda: tf.identity(image))
    image_shape = tf.shape(image)
    
    label = tf.numpy_function(_read_mat, [landmark_dir+name+'_pts.mat', image_shape], tf.float32)
    #label = tf.numpy_function(_read_txt, [landmark_dir+name+'.txt', image_shape], tf.float32)
    
    #image, label = tf.numpy_function(_shrink_image_one, [image, label], [tf.float32, tf.float32])
    #image, label = tf.numpy_function(_crop_image_one, [image, label], [tf.float32, tf.float32])  
    image, label = tf.numpy_function(_downsize_image, [image, label], [tf.float32, tf.float32]) 
    image, label = tf.numpy_function(_shift_image_one, [image, label], [tf.float32, tf.float32])
    image, label = tf.py_function(_flip_image_one, [image, label], [tf.float32, tf.float32])
    
    label = tf.numpy_function(_make_confidence_map, [label, MAP_SIGMA], tf.float32)
    
    image.set_shape([None, None, None])
    image = tf.image.resize(image, [INPUT_SIZE, INPUT_SIZE])
    
    #image.set_shape([INPUT_SIZE, INPUT_SIZE, 3])
    label.set_shape([MAP_SIZE, MAP_SIZE, LANDMARK_SIZE])
    #image = tf.reshape(image, shape=[input_size, input_size, 3])
    #label = tf.reshape(label, shape=[-1])
    
    return image, label

def _read_mat(file_path, input_shape):
    matfile = io.loadmat(file_path)
    norm = [input_shape[1]/2, input_shape[0]/2]
    label = ((matfile['pts_2d']-norm)/norm).astype(np.float32)
    return label

def _read_txt(file_path, input_shape):
    with open(file_path, 'r') as f:
        lines_list = f.readlines()
        temp = list(map(lambda l : list(map(float, l.split(' '))), lines_list[3:-1]))
        temp = np.array(temp)
    norm = [input_shape[1]/2, input_shape[0]/2]
    label = ((temp-norm)/norm).astype(np.float32)
    return label[LANDMARK_NUM]

def _make_confidence_map(label, sigma = 2.5):
    
    norm = [MAP_SIZE/2, MAP_SIZE/2]
    new_label = label*norm+norm
    
    grid_x = np.tile(np.arange(MAP_SIZE), (MAP_SIZE, 1))
    grid_y = np.tile(np.arange(MAP_SIZE), (MAP_SIZE, 1)).transpose()
    grid_x = np.tile(np.expand_dims(grid_x, axis=-1),LANDMARK_SIZE)
    grid_y = np.tile(np.expand_dims(grid_y, axis=-1),LANDMARK_SIZE)
    
    grid_distance = (grid_x - new_label[:,0]) ** 2 + (grid_y - new_label[:,1]) ** 2
    confidence_map = np.exp(-1 * grid_distance / sigma ** 2) # why 0.5?
    
    return confidence_map.astype(np.float32)

def _crop_image_one(img, label): # with label norm
    pad = 1
    
    img_h, img_w, img_c = img.shape
    
    idx = np.array([img_w/2, img_h/2])
    
    label = label*idx+idx
    
    label_x_info = np.array([min(label[:,0]), max(label[:,0])])
    label_y_info = np.array([min(label[:,1]), max(label[:,1])])
    
    ear_w = label_x_info[1]-label_x_info[0]
    ear_h = label_y_info[1]-label_y_info[0]
    
#     s_x = max(int(label_x_info[0]-ear_w*1-pad), 0)
#     e_x = min(int(label_x_info[0]+ear_w*8), img_w)
#     s_y = max(int(label_y_info[0]-ear_h*2-pad), 0)
#     e_y = min(int(label_y_info[0]+ear_h*3), img_h)

    s_x = max(int(label_x_info[0]-ear_w*10), 0)
    e_x = min(int(label_x_info[0]+ear_w*10), img_w)
    s_y = max(int(label_y_info[0]-ear_h*10), 0)
    e_y = min(int(label_y_info[0]+ear_h*10), img_h)

#     s_x = max(int(label_x_info[0]-ear_w*1-pad), 0)
#     e_x = min(int(label_x_info[1]+ear_w*1), img_w)
#     s_y = max(int(label_y_info[0]-ear_h*1-pad), 0)
#     e_y = min(int(label_y_info[1]+ear_h*1), img_h)
    
#     s_x = max(int(label_x_info[0]-pad), 0)
#     e_x = min(int(label_x_info[1]+pad), img_w)
#     s_y = max(int(label_y_info[0]-pad), 0)
#     e_y = min(int(label_y_info[1]+pad), img_h)

    c_img = img[s_y:e_y, s_x:e_x, :]
    c_label = label - np.array([s_x, s_y])
    
    new_img_h, new_img_w, _ = c_img.shape
    
    norm = [new_img_w/2, new_img_h/2]
    c_label = ((c_label-norm)/norm).astype(np.float32)
    return c_img, c_label

def _shrink_image_one(img, label):
    img_h, img_w, img_c = img.shape
    idx = np.array([img_w/2, img_h/2])
    label = label*idx+idx
    
    max_ratio = 4
    sh_ratio = np.random.randint(1,max_ratio)
    
    min_x = int(np.min(label[:,0]))
    max_x = int(np.max(label[:,0]))

    image_left = img[:,0:min_x,:]
    image_right = img[:,max_x:-1,:]
    image_mid = img[:,min_x:max_x:sh_ratio,:]

    sh_img = np.concatenate((image_left, image_mid, image_right), axis = 1)
    sh_label = (label-np.array([min_x,0]))/np.array([sh_ratio,1])+np.array([min_x,0])
    
    new_img_h, new_img_w, _ = sh_img.shape
    norm = [new_img_w/2, new_img_h/2]
    sh_label = ((sh_label-norm)/norm).astype(np.float32)
    
    return sh_img, sh_label
    
def _flip_image_one(img, label):
    c = np.random.randint(2)
    f_img, f_label = tf.cond(c==1,
                             lambda: (tf.image.flip_left_right(img), label * np.array([-1, 1])),
                             lambda: (img, label))
    #f_img = tf.image.resize(f_img, [input_size, input_size])
    return f_img, f_label

def _shift_image_one(img, label, padding = b'zero'):
    img_h, img_w, img_c = img.shape
    label_p, _ = label.shape
    
    if padding == b'ori': s_img = img[:]
    elif padding == b'zero': s_img = np.zeros_like(img, dtype=np.float32)
    s_label = np.expand_dims(label, axis=0)
    
    label_x_info = np.array([min(label[:,0]), max(label[:,0])])* img_w/2 + img_w/2
    label_y_info = np.array([min(label[:,1]), max(label[:,1])])* img_h/2 + img_h/2
    
    label_x_info = label_x_info.astype(np.int)
    label_y_info = label_y_info.astype(np.int)
    
    shift_x = np.random.randint(-label_x_info[0], img_w - label_x_info[1])
    shift_y = np.random.randint(-label_y_info[0], img_h - label_y_info[1])
    
    shift_x = min(shift_x, img_w//6)
    shift_y = min(shift_y, img_h//6)
    
    if shift_x < 0:
        get_x = (-shift_x, img_w)
        put_x = (0, img_w + shift_x)
    else:
        get_x = (0, img_w - shift_x)
        put_x = (shift_x, img_w)
    if shift_y < 0:
        get_y = (-shift_y, img_h)
        put_y = (0, img_h + shift_y)
    else:
        get_y = (0, img_h - shift_y)
        put_y = (shift_y, img_h)

    if padding == b'edge': s_img = np.pad(img[get_y[0]:get_y[1], get_x[0]:get_x[1], :], ((img_h-get_y[1],get_y[0]),(img_w-get_x[1],get_x[0]),(0,0)), mode='edge')
    else: s_img[put_y[0]:put_y[1], put_x[0]:put_x[1], :] = img[get_y[0]:get_y[1], get_x[0]:get_x[1], :]
    s_label = np.append(np.expand_dims(label[:,0] + (shift_x)/(img_w/2), axis = -1),
                          np.expand_dims(label[:,1] + (shift_y)/(img_h/2), axis = -1), axis = 1)
    
    return s_img, s_label

def _downsize_image(img, label):
    img_h, img_w, img_c = img.shape

    size = 1 + np.random.randint(101)/100 # 1~2 (100 step)
    resized_image = cv2.resize(img ,(int(img_w/size), int(img_h/size)))
    #resized_image = tf.image.resize(image ,[int(img_w/size), int(img_h/size)])

    new_img_h, new_img_w, _ = resized_image.shape

    str = np.array([int(img_w/2 * (1 - 1/size)), int(img_h/2 * (1 - 1/size))])
    dst = str + np.array([new_img_w, new_img_h])

    idx = np.array([new_img_w/2, new_img_h/2])
    new_label = (label*idx)+idx + str

    pad_image = np.zeros_like(img, dtype=np.float32)
    pad_image[str[1]:dst[1],str[0]:dst[0],:] = resized_image

    norm = [img_w/2, img_h/2]
    new_label = ((new_label-norm)/norm).astype(np.float32)

    return pad_image, new_label

AUTOTUNE = tf.data.experimental.AUTOTUNE


train_dataset = train_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

test_dataset = test_dataset.map(process_path, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.repeat()
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


for image_raw, label_raw in train_dataset.take(1):
    images = image_raw.numpy()
    labels = label_raw.numpy()
    print(images.shape)
    print(labels.shape)
    
#     norm = np.array([INPUT_SIZE/2, INPUT_SIZE/2])
#     labels = labels.reshape(BATCH_SIZE, -1, 2)
#     labels = labels*norm + norm
    
    for i in range(len(images[:5])):
        
#         for x, y in labels[i]:
#             image_o = cv2.circle(images[i], (round(x), round(y)), 1, (1, 0, 0), -1)
        
        
        heatmap = tf.image.resize(labels[i], (INPUT_SIZE, INPUT_SIZE))
        #mask = np.amax(heatmap, axis=-1, keepdims=True) > 0.1
        mask = heatmap != 0#> 0.0001
        mask = np.amax(mask, axis=-1, keepdims=True)
        print(mask.shape)
        heatmap = cv2.applyColorMap(np.uint8(np.tile(np.amax(heatmap, axis=-1, keepdims=True), 3)*255), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        image_o = images[i]*0.5 + heatmap/255*0.5
        plt.figure(figsize=(10, 10))
        plt.imshow(image_o)#*(mask+0.5)/1.5)
        print(image_o)
        plt.show()
        #print(repr(images[i][:100]))
        #print()
        #print(label_text[i].numpy())

# 학습

In [ ]:
train_step = train_len//BATCH_SIZE
test_step = test_len//BATCH_SIZE

history = model.fit(train_dataset,
                    epochs=EPOCH,
                    steps_per_epoch=train_step,
                    validation_steps=test_step,
                    validation_data=test_dataset,
                    verbose=1)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

In [ ]:
model = tf.keras.models.load_model('saved_model_openpose_face_a2a.h5', compile=False)

# 결과 확인

In [ ]:
ear_hole = np.where(np.array(LANDMARK_NUM) == 36)[0]
ear_edge = np.where(np.array(LANDMARK_NUM) == 9)[0]

pred = tf.keras.backend.function([model.input], [model.get_layer('s6').output])
for image_raw, ture_label in test_dataset.take(3):
    images = image_raw.numpy()
    #labels = model.predict(images) 
    labels = pred([images])[0]
    print(images.shape)
    print(labels.shape)
    for i in range(BATCH_SIZE):
        heatmap = tf.image.resize(labels[i], (INPUT_SIZE, INPUT_SIZE))
        temp = np.argmax(np.reshape(heatmap, (-1,LANDMARK_SIZE)), axis = 0)
        heatmap = cv2.applyColorMap(np.uint8(np.tile(np.amax(heatmap, axis=-1, keepdims=True), 3)*255), cv2.COLORMAP_JET)
        #heatmap = cv2.applyColorMap(np.uint8(np.tile(np.expand_dims(heatmap[:,:,-1], axis = -1), 3)*255), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        image_o = images[i]*0.5 + heatmap/255*0.5
        image_ear = images[i][:]*1
        image_hole = images[i][:]*1
        #plt.imshow(image_o)
        #plt.show()
        
        #temp = np.amax(labels[i], axis=-1)
        #temp = np.argmax(labels[i].reshape(-1,LANDMARK_SIZE), axis = 0)
        #line = [[],[]]
        t_label = tf.image.resize(ture_label[i], (INPUT_SIZE, INPUT_SIZE))
        t_label = np.argmax(np.reshape(t_label, (-1,LANDMARK_SIZE)), axis = 0)
        min_x, max_x = INPUT_SIZE, 0
        min_y, max_y = INPUT_SIZE, 0
        mg = 5
        hole = []
        
        for num, idx in enumerate(temp):
            #x, y = (idx%feat_size)/feat_size, (idx//feat_size)/feat_size
            x, y = idx%INPUT_SIZE, idx//INPUT_SIZE
            t_x, t_y = t_label[num]%INPUT_SIZE, t_label[num]//INPUT_SIZE
            #x *= INPUT_SIZE
            #y *= INPUT_SIZE
            
            min_x, max_x = max(min(int(t_x)-mg, min_x), 0), min(max(int(t_x)+mg, max_x), INPUT_SIZE)
            min_y, max_y = max(min(int(t_y)-mg, min_y), 0), min(max(int(t_y)+mg, max_y), INPUT_SIZE)
            if num == ear_edge or num == ear_hole: hole.append([x, y])
            if num == ear_hole: image_ear = cv2.circle(image_ear, (round(x), round(y)), 1, (0, 1, 0), -1)
            else:        image_ear = cv2.circle(image_ear, (round(x), round(y)), 1, (1, 0, 0), -1)
            image_ear = cv2.circle(image_ear, (int(t_x), int(t_y)), 2, (0, 0, 1), 1)
            image_o = cv2.circle(image_o, (round(x), round(y)), 1, (1, 0, 0), -1)
            #line[0].append(x), line[1].append(y)
        #line[0].append(line[0][0]), line[1].append(line[1][0])
        #np.unravel_index(np.argmax(a, axis=None), a.shape)
        #print(temp.shape, temp)
        #print(loc)
        h_x, h_y = hole[0][0] + (hole[1][0] - hole[0][0])*0.8, hole[0][1] + (hole[1][1] - hole[0][1])*0.8
        image_hole = cv2.circle(image_hole, (int(h_x), int(h_y)), 1, (0, 0, 1), -1)
        
        plt.figure(figsize=(10, 10))
        plt.subplot(1,3,1)
        plt.imshow(image_o)
        #plt.plot(line[0], line[1], '--', linewidth=1, color='firebrick')
        #plt.show()
        #print(min_x, max_x, min_y, max_y)
        plt.subplot(1,3,2)
        plt.imshow(image_ear[min_y:max_y, min_x:max_x,:])
        plt.subplot(1,3,3)
        plt.imshow(image_hole[min_y:max_y, min_x:max_x,:])
        plt.show()

In [ ]:
model.save('saved_model_openpose_face')

In [ ]:
pred = tf.keras.backend.function([model.input], [model.get_layer('s6').output])
for image_raw, ture_label in train_dataset.take(3):
    images = image_raw.numpy()
    #labels = model.predict(images) 
    labels = pred([images])[0]
    print(images.shape)
    print(labels.shape)
    for i in range(BATCH_SIZE):
        heatmap = tf.image.resize(labels[i], (INPUT_SIZE, INPUT_SIZE))
        temp = np.argmax(np.reshape(heatmap, (-1,LANDMARK_SIZE)), axis = 0)
        heatmap = cv2.applyColorMap(np.uint8(np.tile(np.amax(heatmap, axis=-1, keepdims=True), 3)*255), cv2.COLORMAP_JET)
        #heatmap = cv2.applyColorMap(np.uint8(np.tile(np.expand_dims(heatmap[:,:,-1], axis = -1), 3)*255), cv2.COLORMAP_JET)
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
        image_o = images[i]*0.5 + heatmap/255*0.5
        
        for num, idx in enumerate(temp):
            #x, y = (idx%feat_size)/feat_size, (idx//feat_size)/feat_size
            x, y = idx%INPUT_SIZE, idx//INPUT_SIZE
            t_x, t_y = t_label[num]%INPUT_SIZE, t_label[num]//INPUT_SIZE
            #x *= INPUT_SIZE
            #y *= INPUT_SIZE
            
            min_x, max_x = max(min(int(t_x)-mg, min_x), 0), min(max(int(t_x)+mg, max_x), INPUT_SIZE)
            min_y, max_y = max(min(int(t_y)-mg, min_y), 0), min(max(int(t_y)+mg, max_y), INPUT_SIZE)
            if num == ear_edge or num == ear_hole: hole.append([x, y])
            if num == ear_hole: image_ear = cv2.circle(image_ear, (round(x), round(y)), 1, (0, 1, 0), -1)
            else:        image_ear = cv2.circle(image_ear, (round(x), round(y)), 1, (1, 0, 0), -1)
            image_ear = cv2.circle(image_ear, (int(t_x), int(t_y)), 2, (0, 0, 1), 1)
            image_o = cv2.circle(image_o, (round(x), round(y)), 1, (1, 0, 0), -1)

        plt.imshow(image_o)
        plt.show()

In [ ]:
from scipy.ndimage.filters import gaussian_filter

pred = tf.keras.backend.function([model.input], [model.get_layer('s6').output])

for image_raw, ture_label in train_dataset.take(3):
    images = image_raw.numpy()
    heatmaps = pred([images])[0]
    for heatmap in heatmaps:
        all_peaks = []
        peak_counter = 0
        heatmap = heatmap[:,:,0]

        heatmap = gaussian_filter(heatmap, sigma=2.5)
        map_left = np.zeros(heatmap.shape)
        map_right = np.zeros(heatmap.shape)
        map_top = np.zeros(heatmap.shape)
        map_bottom = np.zeros(heatmap.shape)
        map_left[1:, :] = heatmap[:-1, :]
        map_right[:-1, :] = heatmap[1:, :]
        map_top[:, 1:] = heatmap[:, :-1]
        map_bottom[:, :-1] = heatmap[:, 1:]

        peaks_binary = np.logical_and.reduce((
            heatmap > 0.05,
            heatmap > map_left,
            heatmap > map_right,
            heatmap > map_top,
            heatmap > map_bottom,
        ))

        peaks = zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0])  # [(x, y), (x, y)...]
        all_peaks = xp.array([peak for peaks_each_category in all_peaks for peak in peaks_each_category])

        plt.imshow(heatmap)
        plt.show()
        plt.imshow(peaks_binary)
        plt.show()
        print("-------------------------")
